# Data Preprocessing

**This notebook cleans, tokenizes and augments data**

The first part of this notebook applies basic cleaning functions suited for stylometry (preserving stop words and lemmatization).
Then, it applies a French Tokenizer from nltk.

The second part of this notebook augments the data for authors with less than 50 excerpts available. For this it uses controlled LLM generation.
It calls an OpenAI model and uses one-shot in-context learning to guide its generation. Than, the format of the generation is checked, but also it's quality (i.e. how close is it to the original's author style) using Burrow's delta score (a distance metric between vocabulary distributions) to ensure the generation is closer to the target author and isn't an outlier within the existing corpus.

The final output of this notebook is **excerpts_processed.parquet**

May 2025 - Hadrien Strichard

In [1]:
import pandas as pd
import re
import os
from dotenv import load_dotenv
from tqdm import tqdm
import random
import numpy as np
import warnings

# Tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

# Data Augmentation
import openai

# Burrow's Delta Score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cityblock  # Burrows' Delta uses Manhattan distance


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hadrienstrichard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/hadrienstrichard/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Cleaning and tokenizing

In [2]:
# Load initial data

texts = pd.read_parquet('../Data/excerpts_df.parquet')
final_authors = pd.read_csv('../Data/final_authors.csv')

In [3]:
# Basic cleaning function tailored for stylometry (preserving punctuation, no lemmatization)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # remove emails
    text = re.sub(r'\d+', '', text)  # remove digits
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[“”«»]', '"', text)  # normalize quotes
    text = re.sub(r"[’]", "'", text)  # normalize apostrophes
    return text.strip()

In [4]:
# Apply cleaning
texts['Cleaned_Text'] = texts['Excerpt_Text'].apply(clean_text)

In [5]:
# Display first 3 cleaned text for verification
pd.set_option('display.max_colwidth', None)
print(texts['Cleaned_Text'][:3])

0                                                                              n'est souvent déterminée que par un mot. en ce point même, cependant, il n'a fait que se conformer au caprice piquant de la nature, qui se joue à nous faire parcourir dans la durée d'un seul rêve, plusieurs fois interrompu par des épisodes étrangers à son objet, tous les développements d'une action régulière, complète et plus ou moins vraisemblable. les personnes qui ont lu apulée s'apercevront facilement que la fable du premier livre de l'_âne d'or_ de cet ingénieux conteur a beaucoup de rapports avec celle-ci, et qu'elles se ressemblent par le fond presque autant qu'elles diffèrent par la forme. l'auteur paraît même avoir affecté de solliciter ce rapprochement en conservant à son principal personnage le nom de lucius. le récit du philosophe de madaure et celui du prêtre dalmate, cité par fortis, tome i, page , ont en effet une origine commune dans les chants traditionnels d'une contrée qu'apulée avait curi

In [6]:
# Load French tokenizer
punkt_param = PunktParameters()
tokenizer = PunktSentenceTokenizer(punkt_param)

In [7]:
# Apply word tokenization
texts['Tokens'] = texts['Cleaned_Text'].apply(lambda x: word_tokenize(x, language='french'))

In [8]:
# Display first 3 tokenized text for verification
print(texts['Tokens'][:3])

0                                                                     [n'est, souvent, déterminée, que, par, un, mot, ., en, ce, point, même, ,, cependant, ,, il, n, ', a, fait, que, se, conformer, au, caprice, piquant, de, la, nature, ,, qui, se, joue, à, nous, faire, parcourir, dans, la, durée, d'un, seul, rêve, ,, plusieurs, fois, interrompu, par, des, épisodes, étrangers, à, son, objet, ,, tous, les, développements, d'une, action, régulière, ,, complète, et, plus, ou, moins, vraisemblable, ., les, personnes, qui, ont, lu, apulée, s'apercevront, facilement, que, la, fable, du, premier, livre, de, l'_âne, d'or_, de, cet, ingénieux, conteur, a, beaucoup, de, rapports, avec, celle-ci, ,, et, qu'elles, se, ...]
1                                                                                                              [le, reste, ne, me, regarde, point, ., j'ai, dit, de, qui, était, la, fable, :, sauf, quelques, phrases, de, transition, ,, tout, appartient, à, homère, ,, à, théocrite,

In [9]:
# Drop authors and excerpts with < 25 excerpts
author_counts = texts['Author'].value_counts()
valid_authors = author_counts[author_counts > 25].index
texts = texts[texts['Author'].isin(valid_authors)]

# Filter final_authors
final_authors = final_authors[final_authors['Name'].isin(valid_authors)]

# Update books.csv
books = pd.read_csv('../Data/books.csv')
books = books[books['Author'].isin(valid_authors)]
books.to_csv('../Data/books.csv', index=False)

# Save updated data
texts.to_parquet('../Data/excerpts_processed.parquet')
final_authors.to_csv('../Data/final_authors.csv', index=False)

In [10]:
final_authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        28 non-null     object
 1   birth_date  28 non-null     object
 2   death_date  28 non-null     object
 3   n_books     28 non-null     int64 
 4   Mouvement   28 non-null     object
 5   Genres      28 non-null     object
 6   n_excerpts  28 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 1.7+ KB


In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Author     299 non-null    object 
 1   Title      299 non-null    object 
 2   Language   299 non-null    object 
 3   EBook-No   299 non-null    int64  
 4   URL        299 non-null    object 
 5   Roman      299 non-null    bool   
 6   Genres     299 non-null    object 
 7   Mouvement  299 non-null    object 
 8   Date       298 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 19.1+ KB


In [12]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14722 entries, 0 to 14754
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Author        14722 non-null  object
 1   Title         14722 non-null  object
 2   URL           14722 non-null  object
 3   Excerpt_ID    14722 non-null  object
 4   Excerpt_Text  14722 non-null  object
 5   Cleaned_Text  14722 non-null  object
 6   Tokens        14722 non-null  object
dtypes: object(7)
memory usage: 920.1+ KB


## Data Aug

In [13]:
# Load OpenAI key

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

Delta Score is based on Burrow's Delta metric, a distance metric between texts based on vocabulary distribution

In [14]:
# Delta Score Calculation

def compute_author_profiles(texts_df):
    """Creates stylometric profiles per author"""
    vectorizer = CountVectorizer(max_features=300, analyzer='word', lowercase=True)
    X = vectorizer.fit_transform(texts_df["Cleaned_Text"])
    features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    features["Author"] = texts_df["Author"].values

    # Compute mean frequencies per author
    author_profiles = features.groupby("Author").mean()

    # Z-score normalization (feature-wise)
    scaler = StandardScaler()
    scaled_profiles = pd.DataFrame(
        scaler.fit_transform(author_profiles),
        index=author_profiles.index,
        columns=author_profiles.columns
    )
    return scaled_profiles, scaler, vectorizer

def delta_score(text, scaled_author_profiles, scaler, vectorizer):
    """Compute Delta between one excerpt and all author profiles"""
    vec = vectorizer.transform([text])
    vec_df = pd.DataFrame(vec.toarray(), columns=vectorizer.get_feature_names_out())

    # Align columns
    for col in scaled_author_profiles.columns:
        if col not in vec_df.columns:
            vec_df[col] = 0
    vec_df = vec_df[scaled_author_profiles.columns]

    # Normalize using same scaler as author profiles
    vec_scaled = pd.DataFrame(
        scaler.transform(vec_df),
        columns=vec_df.columns
    ).iloc[0]

    # Compute Manhattan (L1) distance: Burrows’ Delta
    deltas = {}
    for author in scaled_author_profiles.index:
        profile = scaled_author_profiles.loc[author]
        deltas[author] = cityblock(vec_scaled, profile)

    return deltas


def get_delta_distribution_for_author(author, texts_df, profiles, scaler, vectorizer):
    subset = texts_df[texts_df["Author"] == author]["Cleaned_Text"]
    deltas = [delta_score(text, profiles, scaler, vectorizer)[author] for text in subset]
    return deltas

The prompt is in French for a generation in French. It uses one-shot learning (i.e. an example is given) and a context (given an excerpt predict next one)

In [15]:
def build_prompt(target_author, example_1, example_2, input_excerpt, author_metadata):
    """Given an author, two examples, and an excerpt, builds a prompt for text generation."""
    bio = (
        f"{target_author} est un écrivain français du {author_metadata['birth_date'][:4]}–{author_metadata['death_date'][:4]} siècle. "
        f"Appartenant au mouvement {author_metadata['Mouvement']}, il a principalement écrit dans les genres suivants : "
        f"{author_metadata['Genres']}."
    )

    return f"""
Tu es un modèle de génération de texte littéraire spécialisé dans l'augmentation de données pour la classification d'auteurs.

Ton rôle :
- Générer des extraits de fiction de 1500 mots (+/- 10%) en français.
- Respecter scrupuleusement le style de l’auteur : vocabulaire, syntaxe, ponctuation, ton, époque.
- Ne pas inclure de dialogues, titres ou en-têtes : que du texte narratif ou descriptif.
- Générer un passage qui suit naturellement celui donné, comme s’il s’agissait de la suite immédiate dans le livre.

Informations :
{bio}

Exemple de suite entre deux extraits d’un même livre de cet auteur:
--- Extrait A ---
{example_1}
--- Extrait B (suite de A) ---
{example_2}

Tâche :
Voici un extrait tiré de l’œuvre suivante : "{input_excerpt['Title']}". Génére la suite immédiate de cet extrait dans le même style, sans introduire de coupure ou de résumé.

--- Extrait donné ---
{input_excerpt['Excerpt_Text']}
--- Ta réponse (suite de l'extrait) ---
"""


In [16]:
def generate_and_validate(author_row, texts_df, n_needed, author_profiles, vectorizer, scaler, client):
    """Genrates and validates excerpts for a given author."""

    author_name = author_row["Name"]
    subset = texts_df[texts_df["Author"] == author_name].copy()

    # Extract book ID and position
    subset[["Book_ID", "Book_Pos"]] = subset["Excerpt_ID"].str.extract(r"(\d+)_(\d+)")
    subset["Book_Pos"] = subset["Book_Pos"].astype(int)

    # For each book, keep middle 50% of excerpts
    middle_excerpts = []
    for book_id, group in subset.groupby("Book_ID"):
        group_sorted = group.sort_values("Book_Pos")
        n = len(group_sorted)
        if n >= 4:
            middle = group_sorted.iloc[n // 4: 3 * n // 4]
            middle_excerpts.append(middle)
        else:
            # If too few excerpts, keep them all (fallback)
            middle_excerpts.append(group_sorted)

    middle_subset = pd.concat(middle_excerpts).reset_index(drop=True)


    new_excerpts = []
    attempts = 0
    max_attempts = 5 * n_needed 

    while len(new_excerpts) < n_needed and attempts < max_attempts:
        attempts += 1

        # Random consecutive excerpts for example
        examples = middle_subset.sample(2).sort_values("Excerpt_ID")
        example_1, example_2 = examples.iloc[0]["Excerpt_Text"], examples.iloc[1]["Excerpt_Text"]

        # Input excerpt also from middle
        input_excerpt = middle_subset.sample(1).iloc[0]
        
        # Build the prompt
        prompt = build_prompt(
            target_author=author_name,
            example_1=example_1,
            example_2=example_2,
            input_excerpt=input_excerpt,
            author_metadata=author_row
        )

        # Validation process
        try:
            print(f"Generating excerpt for {author_name}... (Attempt {attempts})")
            response = client.chat.completions.create(
                model="gpt-4.1-2025-04-14",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
                max_tokens=2048
            )
            generated = response.choices[0].message.content
            print(f"Generated excerpt for {author_name}: {generated[:100]}...")

            cleaned_gen = clean_text(generated)
            tokenized_gen = word_tokenize(cleaned_gen, language='french')
            print(len(tokenized_gen), "tokens in generated excerpt")
            length_ok = 1350 <= len(tokenized_gen) <= 1650

            delta_scores = delta_score(cleaned_gen, author_profiles, scaler, vectorizer)
            print(f"Delta scores for {author_name}: {delta_scores}")
            sorted_authors = sorted(delta_scores.items(), key=lambda x: x[1])
            closest_author, closest_score = sorted_authors[0]
            target_score = delta_scores[author_name]
            print(f"Closest author: {closest_author} ({closest_score:.2f}), Target author ({author_name}): {target_score:.2f}")

            # Allow small tolerance if author is nearly the closest
            tolerance = attempts / 100  # Make the tolerance increase with attempts
            delta_gap = abs(closest_score - target_score)
            is_closest_or_near = (
                closest_author == author_name or
                (target_score - closest_score) / closest_score <= tolerance
            )

            # Delta range check
            real_deltas = get_delta_distribution_for_author(
                author_name, texts_df, author_profiles, scaler, vectorizer
            )
            threshold = np.percentile(real_deltas, 90)
            within_range = target_score <= threshold

            print(f"Delta to {author_name}: {target_score:.2f} | 90th percentile: {threshold:.2f}")
            good_style = is_closest_or_near and within_range
            print(f"Excerpt length OK: {length_ok}, Good style: {good_style}")

            if length_ok and good_style:
                new_excerpts.append({
                    "Author": author_name,
                    "Title": input_excerpt["Title"],
                    "URL": input_excerpt["URL"],
                    "Excerpt_ID": f"{input_excerpt['Excerpt_ID']}_gen{len(new_excerpts)}",
                    "Excerpt_Text": generated,
                    "Cleaned_Text": cleaned_gen,
                    "Tokens": tokenized_gen,
                    "of_which_generated": input_excerpt["Excerpt_ID"]
                })
                print(f"✅ Accepted excerpt for {author_name}: {len(new_excerpts)} / {n_needed}")

        except Exception as e:
            print(f"Error (attempt {attempts}):", e)
            continue

    if len(new_excerpts) < n_needed:
        warnings.warn(
            f"⚠️ Could not generate all excerpts for {author_name}. Only {len(new_excerpts)} out of {n_needed} were accepted. Review needed."
        )

    return pd.DataFrame(new_excerpts)


In [17]:
# Precompute author profiles
author_profiles, scaler, vectorizer = compute_author_profiles(texts)

# Get underrepresented authors
need_aug = final_authors[final_authors["n_excerpts"] < 50]

print(f"Authors needing augmentation: {len(need_aug)}")

Authors needing augmentation: 3


In [18]:
# Generate and validate excerpts for each author needing augmentation

augmented_all = []

client = client = openai.OpenAI()

for _, author_row in tqdm(need_aug.iterrows(), total=need_aug.shape[0]):
    n_needed = 50 - author_row["n_excerpts"]
    df_aug = generate_and_validate(author_row, texts, n_needed, author_profiles, vectorizer, scaler, client)
    augmented_all.append(df_aug)

augmented_df = pd.concat(augmented_all, ignore_index=True)

  0%|          | 0/3 [00:00<?, ?it/s]

Generating excerpt for Charles Nodier... (Attempt 1)
Generated excerpt for Charles Nodier: L’attention de l’assemblée, d’abord flottante et bruyante, se fixa peu à peu sur Lothario, comme si ...
1441 tokens in generated excerpt
Delta scores for Charles Nodier: {'Alain-Fournier': np.float64(642.7019864681065), 'Alexandre Dumas': np.float64(651.5062476436663), 'Alfred de Vigny': np.float64(653.0846911827723), 'Alphonse Daudet': np.float64(595.9718365020294), 'Anatole France': np.float64(592.6470346613145), 'André Gide': np.float64(628.3640288891745), 'Charles Nodier': np.float64(529.2898501859817), 'François Mauriac': np.float64(553.5249823824644), 'George Sand': np.float64(668.1183072319594), 'Georges Bernanos': np.float64(620.8681126248757), 'Gustave Flaubert': np.float64(580.9752568796837), 'Guy de Maupassant': np.float64(600.6163065187284), 'Henri Barbusse': np.float64(642.2281534625049), 'Honoré de Balzac': np.float64(685.7131116519186), 'Jules Renard': np.float64(618.7725167939864)

 33%|███▎      | 1/3 [26:14<52:28, 1574.44s/it]

Generated excerpt for Charles Nodier: On disait de moi, dans les veillées du village, des choses que je n’aurais pas reconnues pour mienne...
1574 tokens in generated excerpt
Delta scores for Charles Nodier: {'Alain-Fournier': np.float64(628.4483075740585), 'Alexandre Dumas': np.float64(679.8087931784091), 'Alfred de Vigny': np.float64(684.1451110665548), 'Alphonse Daudet': np.float64(639.2844564481658), 'Anatole France': np.float64(644.7304168475142), 'André Gide': np.float64(610.4305194447106), 'Charles Nodier': np.float64(569.4319816394777), 'François Mauriac': np.float64(575.2647370063848), 'George Sand': np.float64(661.1437395363947), 'Georges Bernanos': np.float64(656.1890161676182), 'Gustave Flaubert': np.float64(629.4305025058568), 'Guy de Maupassant': np.float64(627.1620537025851), 'Henri Barbusse': np.float64(680.7968994225582), 'Honoré de Balzac': np.float64(692.216232110417), 'Jules Renard': np.float64(625.3013554359939), 'Jules Vallès': np.float64(695.8311240351421), 'Jule

 67%|██████▋   | 2/3 [48:04<23:38, 1418.91s/it]

Generated excerpt for Jules Renard: Je m’efforce de sourire, de me donner un air badin, mais la nuit me coule sur les épaules, et la mer...
1635 tokens in generated excerpt
Delta scores for Jules Renard: {'Alain-Fournier': np.float64(680.4006951383308), 'Alexandre Dumas': np.float64(710.7830910178295), 'Alfred de Vigny': np.float64(706.1167048194566), 'Alphonse Daudet': np.float64(657.8436831415738), 'Anatole France': np.float64(675.3652373418429), 'André Gide': np.float64(653.2402902584242), 'Charles Nodier': np.float64(657.6687792713499), 'François Mauriac': np.float64(644.9839216825754), 'George Sand': np.float64(713.7903745722894), 'Georges Bernanos': np.float64(692.7011746158154), 'Gustave Flaubert': np.float64(659.7498270608805), 'Guy de Maupassant': np.float64(666.520939780502), 'Henri Barbusse': np.float64(671.0163576294772), 'Honoré de Balzac': np.float64(745.1560137409647), 'Jules Renard': np.float64(597.0399927480372), 'Jules Vallès': np.float64(696.5723510642429), 'Jules Ve

100%|██████████| 3/3 [1:19:04<00:00, 1581.35s/it]

Generated excerpt for Alain-Fournier: Ce soir-là, dans la petite salle assombrie, la flamme vacillante de la bougie projetait sur les murs...
1583 tokens in generated excerpt
Delta scores for Alain-Fournier: {'Alain-Fournier': np.float64(600.1491877653995), 'Alexandre Dumas': np.float64(701.8530351657552), 'Alfred de Vigny': np.float64(697.4169318558602), 'Alphonse Daudet': np.float64(639.6534747326978), 'Anatole France': np.float64(670.3847158930004), 'André Gide': np.float64(634.9759838870418), 'Charles Nodier': np.float64(659.1641791992549), 'François Mauriac': np.float64(593.7621298603867), 'George Sand': np.float64(704.3613167498715), 'Georges Bernanos': np.float64(657.9781873703135), 'Gustave Flaubert': np.float64(652.4809484931005), 'Guy de Maupassant': np.float64(648.0385730276188), 'Henri Barbusse': np.float64(685.4980533998805), 'Honoré de Balzac': np.float64(717.0311775293848), 'Jules Renard': np.float64(678.221376356506), 'Jules Vallès': np.float64(702.9518347858124), 'Jule

### Exploration, processing and saving of generated texts

In [19]:
pd.set_option('display.max_colwidth', 50)
augmented_df

,Author,Title,URL,Excerpt_ID,Excerpt_Text,Cleaned_Text,Tokens,of_which_generated
0,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_12_gen0,"L’attention de l’assemblée, d’abord flottante ...","l'attention de l'assemblée, d'abord flottante ...","[l'attention, de, l'assemblée, ,, d'abord, flo...",64289_12
1,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_3_gen1,"Car, dans ce sommeil où je m’abîme, il n’est p...","car, dans ce sommeil où je m'abîme, il n'est p...","[car, ,, dans, ce, sommeil, où, je, m'abîme, ,...",18083_3
2,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_19_gen2,"Je vécus ainsi plusieurs semaines, livré à la ...","je vécus ainsi plusieurs semaines, livré à la ...","[je, vécus, ainsi, plusieurs, semaines, ,, liv...",64289_19
3,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_13_gen3,"C’est ainsi que, dans la retraite paisible de ...","c'est ainsi que, dans la retraite paisible de ...","[c'est, ainsi, que, ,, dans, la, retraite, pai...",64289_13
4,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_9_gen4,"On conte encore, à la veillée, sous les toits ...","on conte encore, à la veillée, sous les toits ...","[on, conte, encore, ,, à, la, veillée, ,, sous...",64289_9
5,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_13_gen5,Ce fut dans cette disposition d’esprit que Mme...,ce fut dans cette disposition d'esprit que mme...,"[ce, fut, dans, cette, disposition, d'esprit, ...",64289_13
6,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_18_gen6,"Mais le désespoir, ce mal sans remède, s’insin...","mais le désespoir, ce mal sans remède, s'insin...","[mais, le, désespoir, ,, ce, mal, sans, remède...",64289_18
7,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_10_gen7,"Matteo, dont la figure ridée et la parole lent...","matteo, dont la figure ridée et la parole lent...","[matteo, ,, dont, la, figure, ridée, et, la, p...",64289_10
8,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_14_gen8,"Antonia, interdite, demeura un instant immobil...","antonia, interdite, demeura un instant immobil...","[antonia, ,, interdite, ,, demeura, un, instan...",64289_14
9,Charles Nodier,Jean Sbogar,https://www.gutenberg.org/ebooks/64289,64289_19_gen9,"Je poursuivis ainsi, livré à la misanthropie d...","je poursuivis ainsi, livré à la misanthropie d...","[je, poursuivis, ainsi, ,, livré, à, la, misan...",64289_19


In [31]:
max_length = 5000

for author in augmented_df["Author"].unique():
    subset = augmented_df[augmented_df["Author"] == author].copy()

    subset["Delta_Score"] = subset["Cleaned_Text"].apply(
        lambda text: delta_score(text, author_profiles, scaler, vectorizer)[author]
    )

    subset_sorted = subset.sort_values("Delta_Score")
    best = subset_sorted.iloc[0]
    worst = subset_sorted.iloc[-1]

    best_input = texts[texts["Excerpt_ID"] == best["of_which_generated"]]
    worst_input = texts[texts["Excerpt_ID"] == worst["of_which_generated"]]

    print("=" * 80)
    print(f"AUTEUR : {author}")
    
    print("\n--- MEILLEUR EXTRAIT ---")
    if not best_input.empty:
        print(f"ID extrait d'origine : {best_input.iloc[0]['Excerpt_ID']}")
        print(f"Texte original :\n{best_input.iloc[0]['Excerpt_Text']}...\n")
    print(f"ID : {best['Excerpt_ID']}")
    print(f"Score delta : {best['Delta_Score']:.4f}")
    print(f"Texte généré :\n{best['Excerpt_Text']}...\n")

    print("\n--- PIRE EXTRAIT ---")
    if not worst_input.empty:
        print(f"ID extrait d'origine : {worst_input.iloc[0]['Excerpt_ID']}")
        print(f"Texte original :\n{worst_input.iloc[0]['Excerpt_Text']}...\n")
    print(f"ID : {worst['Excerpt_ID']}")
    print(f"Score delta : {worst['Delta_Score']:.4f}")
    print(f"Texte généré :\n{worst['Excerpt_Text']}...\n")


AUTEUR : Charles Nodier

--- MEILLEUR EXTRAIT ---
ID extrait d'origine : 64289_13
Texte original :
Antonia, vivement émue par le choix de cet air et par le son de la
voix de Lothario, se rapprocha de madame Alberti, qui était très
préoccupée de son côté. Elle se rappelait aussi cette voix
harmonieuse et le lieu où elle l’avait entendue; mais ce pouvait
être l’effet d’une ressemblance fortuite. Le chant dalmate est
trop simple, trop uniforme, trop dépouillé d’ornements, pour qu’il
ne soit pas aisé de se méprendre entre deux voix analogues. Enfin,
après un moment de réflexion, Lothario reprit sa romance tout
entière, en continuant à s’accompagner de ces accords aériens que
la harpe rendait sous ses doigts, et dont la mélodie religieuse se
mariait avec son chant de la manière la plus imposante. Parvenu au
refrain du vieux Morlaque, il y mit l’accent d’une pitié si
douloureuse que tous les cœurs en furent attendris, mais surtout
celui d’Antonia, qui attachait à cette idée un souvenir
d’inq

In [21]:
# Ajoute la colonne Delta pour les statistiques
augmented_df["Delta_Score"] = augmented_df.apply(
    lambda row: delta_score(row["Cleaned_Text"], author_profiles, scaler, vectorizer)[row["Author"]],
    axis=1
)

# Token count
augmented_df["Token_Count"] = augmented_df["Tokens"].apply(len)

validation_stats = {
    "Total excerpts generated": len(augmented_df),
    "Number of authors": augmented_df["Author"].nunique(),
    "Average token count": round(augmented_df["Token_Count"].mean(), 2),
    "Min token count": augmented_df["Token_Count"].min(),
    "Max token count": augmented_df["Token_Count"].max(),
    "Average delta score": round(augmented_df["Delta_Score"].mean(), 4),
    "Min delta score": round(augmented_df["Delta_Score"].min(), 4),
    "Max delta score": round(augmented_df["Delta_Score"].max(), 4),
}

# Affichage clair
print("\n📊 Statistiques de validation :")
for k, v in validation_stats.items():
    print(f"- {k}: {v}")



📊 Statistiques de validation :
- Total excerpts generated: 37
- Number of authors: 3
- Average token count: 1512.16
- Min token count: 1389
- Max token count: 1635
- Average delta score: 577.2265
- Min delta score: 522.9459
- Max delta score: 613.4699


In [22]:
# Check tokenization consistency of the generated excerpts
print(augmented_df["Tokens"].apply(type).value_counts())

# Check IDs of the generated excerpts
pattern = re.compile(r"\d+_\d+_gen\d+")
invalid_ids = augmented_df[~augmented_df["Excerpt_ID"].str.match(pattern)]
print(f"{len(invalid_ids)} IDs don't have a valide format.")

Tokens
<class 'list'>    37
Name: count, dtype: int64
0 IDs don't have a valide format.


In [23]:
# Add new excerpts and update texts DataFrame


texts = pd.concat([texts, augmented_df], ignore_index=True)

# Update author excerpt count
updated_counts = texts["Author"].value_counts().reset_index()
updated_counts.columns = ["Name", "n_excerpts"]
final_authors.update(updated_counts)

# Fill missing "of_which_generated" for real texts
texts["of_which_generated"] = texts.get("of_which_generated", np.nan)

In [24]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14759 entries, 0 to 14758
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Author              14759 non-null  object 
 1   Title               14759 non-null  object 
 2   URL                 14759 non-null  object 
 3   Excerpt_ID          14759 non-null  object 
 4   Excerpt_Text        14759 non-null  object 
 5   Cleaned_Text        14759 non-null  object 
 6   Tokens              14759 non-null  object 
 7   of_which_generated  37 non-null     object 
 8   Delta_Score         37 non-null     float64
 9   Token_Count         37 non-null     float64
dtypes: float64(2), object(8)
memory usage: 1.1+ MB


In [25]:
pd.set_option('display.max_colwidth', 50)
texts.head()

,Author,Title,URL,Excerpt_ID,Excerpt_Text,Cleaned_Text,Tokens,of_which_generated,Delta_Score,Token_Count
0,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_1,n'est souvent déterminée que par un mot. En ce...,n'est souvent déterminée que par un mot. en ce...,"[n'est, souvent, déterminée, que, par, un, mot...",NaN,NaN,NaN
1,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_2,Le reste ne me regarde point. J'ai dit de qui ...,le reste ne me regarde point. j'ai dit de qui ...,"[le, reste, ne, me, regarde, point, ., j'ai, d...",NaN,NaN,NaN
2,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_3,"Les sylphes, tout étourdis du bruit de la veil...","les sylphes, tout étourdis du bruit de la veil...","[les, sylphes, ,, tout, étourdis, du, bruit, d...",NaN,NaN,NaN
3,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_4,"À peine mes yeux sont fermés, à peine cesse la...","à peine mes yeux sont fermés, à peine cesse la...","[à, peine, mes, yeux, sont, fermés, ,, à, pein...",NaN,NaN,NaN
4,Charles Nodier,Smarra ou les démons de la nuit: Songes romant...,https://www.gutenberg.org/ebooks/18083,18083_5,"C'est en vain que le jour s'éteindrait, tant q...","c'est en vain que le jour s'éteindrait, tant q...","[c'est, en, vain, que, le, jour, s'éteindrait,...",NaN,NaN,NaN


In [26]:
pd.set_option('display.max_colwidth', 500000000)

print(texts[texts["Excerpt_ID"]=='64289_14_gen0']['Excerpt_Text'])


Series([], Name: Excerpt_Text, dtype: object)


In [32]:
texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14759 entries, 0 to 14758
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Author              14759 non-null  object 
 1   Title               14759 non-null  object 
 2   URL                 14759 non-null  object 
 3   Excerpt_ID          14759 non-null  object 
 4   Excerpt_Text        14759 non-null  object 
 5   Cleaned_Text        14759 non-null  object 
 6   Tokens              14759 non-null  object 
 7   of_which_generated  37 non-null     object 
 8   Delta_Score         37 non-null     float64
 9   Token_Count         37 non-null     float64
dtypes: float64(2), object(8)
memory usage: 1.1+ MB


In [27]:
# Save final versions
# texts.to_parquet('../Data/excerpts_processed.parquet', index=False)
# final_authors.to_csv('../Data/final_authors.csv', index=False)
